In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import datetime

## Import data

In [ ]:
df = pd.read_csv('/content/2021-2022_bit.csv')

In [ ]:
df = df.iloc[:,1:].copy()

In [ ]:
# reverse the data
df = df[::-1]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226079 entries, 226078 to 0
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Date        226079 non-null  object 
 1   Open        226079 non-null  float64
 2   High        226079 non-null  float64
 3   Low         226079 non-null  float64
 4   Close       226079 non-null  float64
 5   Volume BTC  226079 non-null  float64
dtypes: float64(5), object(1)
memory usage: 10.3+ MB


In [ ]:
df.columns = ['date', 'open', 'high', 'low', 'close', 'vol']

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df = df.set_index('date')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 226079 entries, 2021-08-01 00:01:00 to 2022-01-04 23:59:00
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    226079 non-null  float64
 1   high    226079 non-null  float64
 2   low     226079 non-null  float64
 3   close   226079 non-null  float64
 4   vol     226079 non-null  float64
dtypes: float64(5)
memory usage: 10.3 MB


In [ ]:
df[df.isnull()]

,open,high,low,close,vol
date,,,,,
2021-08-01 00:01:00,NaN,NaN,NaN,NaN,NaN
2021-08-01 00:02:00,NaN,NaN,NaN,NaN,NaN
2021-08-01 00:03:00,NaN,NaN,NaN,NaN,NaN
2021-08-01 00:04:00,NaN,NaN,NaN,NaN,NaN
2021-08-01 00:05:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-01-04 23:55:00,NaN,NaN,NaN,NaN,NaN
2022-01-04 23:56:00,NaN,NaN,NaN,NaN,NaN
2022-01-04 23:57:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])

In [ ]:
df_aligned = df.loc[df_diff.index]

In [ ]:
# Train data
# Period : From start of 2018 to end of 2022
mask_train = (df_diff.index >= "2021-10-01") & (df_diff.index < "2021-12-31")
df_train = df_diff.loc[mask_train].copy()
train_close = df_aligned.loc[mask_train, "close"].values
df_train["Relative_Close"] = train_close / train_close[0]

In [ ]:
# Test data
# Period : Whole 2023
mask_test = (df_diff.index >= "2021-12-31") & (df_diff.index < "2022-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]


In [ ]:
df_train.head()

,open,high,low,close,vol,Relative_Close
date,,,,,,
2021-10-01 00:00:00,49.39,0.00,35.28,0.00,0.172721,1.000000
2021-10-01 00:01:00,0.00,43.23,14.11,41.79,0.138222,1.000953
2021-10-01 00:02:00,41.79,85.12,-27.83,86.56,1.004677,1.002928
2021-10-01 00:03:00,86.56,16.26,127.39,14.35,2.271787,1.003256
2021-10-01 00:04:00,14.35,12.35,24.08,14.26,1.217967,1.003581


In [ ]:
df_test.head()

,open,high,low,close,vol,Relative_Close
date,,,,,,
2021-12-31 00:00:00,-8.94,-23.58,-30.65,-30.65,0.108346,1.074458
2021-12-31 00:01:00,-30.65,67.03,0.00,95.45,0.035732,1.076636
2021-12-31 00:02:00,95.45,76.63,88.21,72.93,0.090812,1.078300
2021-12-31 00:03:00,72.93,19.19,27.12,-53.05,0.887851,1.077090
2021-12-31 00:04:00,-53.05,-58.71,-2.89,19.46,0.722145,1.077534


In [ ]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

### LAG = 5

In [ ]:
LAG = 5

In [ ]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
# Model architecture
tf.keras.utils.set_random_seed(4002)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20
3276/3276 [==============================] - 19s 5ms/step - loss: 3046.9412 - val_loss: 1557.3982
Epoch 2/20
3276/3276 [==============================] - 16s 5ms/step - loss: 3041.7810 - val_loss: 1559.9789
Epoch 3/20
3276/3276 [==============================] - 16s 5ms/step - loss: 3036.5896 - val_loss: 1555.1101
Epoch 4/20
3276/3276 [==============================] - 18s 6ms/step - loss: 3037.6938 - val_loss: 1556.5785
Epoch 5/20
3276/3276 [==============================] - 17s 5ms/step - loss: 3035.2117 - val_loss: 1557.4156
Epoch 6/20
3276/3276 [==============================] - 16s 5ms/step - loss: 3009.2217 - val_loss: 1559.2483
Epoch 7/20
3276/3276 [==============================] - 19s 6ms/step - loss: 3010.4485 - val_loss: 1557.1094
Epoch 8/20
3276/3276 [==============================] - 17s 5ms/step - loss: 3023.2427 - val_loss: 1555.3612
Epoch 9/20
3276/3276 [==============================] - 16s 5ms/step - loss: 3022.6052 - val_loss: 1553.1786
Epoch 10/20
3276/32

In [ ]:
# Predict
predictions = model.predict(X_test)

45/45 [==============================] - 0s 3ms/step


In [ ]:
# Evaluate
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f'Test MSE: {mse}')

Test MSE: 1447.7574393095827


### LAG = 50

In [ ]:
LAG = 50

In [ ]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
# Model architecture
tf.keras.utils.set_random_seed(4002)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20
3275/3275 [==============================] - 85s 25ms/step - loss: 5881704.5000 - val_loss: 4459.2368
Epoch 2/20
3275/3275 [==============================] - 75s 23ms/step - loss: 400093708288.0000 - val_loss: 11526.9014
Epoch 3/20
3275/3275 [==============================] - 75s 23ms/step - loss: 452758.2500 - val_loss: 101430.3594
Epoch 4/20
3275/3275 [==============================] - 79s 24ms/step - loss: 267479.9062 - val_loss: 4148.8315
Epoch 5/20
3275/3275 [==============================] - 75s 23ms/step - loss: 14822.4629 - val_loss: 4100.6479
Epoch 6/20
3275/3275 [==============================] - 77s 23ms/step - loss: 11788.5371 - val_loss: 3499.8462
Epoch 7/20
3275/3275 [==============================] - 75s 23ms/step - loss: 18548.2207 - val_loss: 2773.3567
Epoch 8/20
3275/3275 [==============================] - 76s 23ms/step - loss: 3864.5286 - val_loss: 3013.7363
Epoch 9/20
3275/3275 [==============================] - 76s 23ms/step - loss: 3294.5945 - val_loss:

In [ ]:
# Predict
predictions = model.predict(X_test)

45/45 [==============================] - 1s 8ms/step


In [ ]:
# Evaluate
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f'Test MSE: {mse}')

Test MSE: 1586.4653692194706


In [ ]:
LAG = 3

In [ ]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
# Model architecture
tf.keras.utils.set_random_seed(4002)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20
3276/3276 [==============================] - 19s 5ms/step - loss: 3039.3218 - val_loss: 1554.2931
Epoch 2/20
3276/3276 [==============================] - 13s 4ms/step - loss: 3034.7227 - val_loss: 1556.8706
Epoch 3/20
3276/3276 [==============================] - 13s 4ms/step - loss: 3025.0264 - val_loss: 1559.4854
Epoch 4/20
3276/3276 [==============================] - 13s 4ms/step - loss: 3013.3279 - val_loss: 1557.5371
Epoch 5/20
3276/3276 [==============================] - 13s 4ms/step - loss: 3021.2957 - val_loss: 1556.1709
Epoch 6/20
3276/3276 [==============================] - 15s 4ms/step - loss: 3017.9131 - val_loss: 1557.9049
Epoch 7/20
3276/3276 [==============================] - 14s 4ms/step - loss: 3011.8406 - val_loss: 1553.6736
Epoch 8/20
3276/3276 [==============================] - 15s 5ms/step - loss: 3012.8276 - val_loss: 1562.1426
Epoch 9/20
3276/3276 [==============================] - 14s 4ms/step - loss: 3003.8630 - val_loss: 1560.0449
Epoch 10/20
3276/32

In [ ]:
# Predict
predictions = model.predict(X_test)

45/45 [==============================] - 0s 2ms/step


In [ ]:
# Evaluate
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f'Test MSE: {mse}')

Test MSE: 1457.6206009779517
